In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd '/content/drive/My Drive/intern/' 

/content/drive/My Drive/intern


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as scipy
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
import fancyimpute as fancyimpute

In [0]:
cbd = pd.read_csv('./input/Customer Bill Detail.csv', header = 0)
coid = pd.read_csv('./input/Customer Order Item Details.csv', header = 0)
mp = pd.read_csv('./input/Market Pricing.csv', header = 0)

In [0]:
#Biavariate Cat-Cat Two Way table
def bi_cat_cat_2way(label1, label2, data):
  two_way = {}
  two_way['normal'] = pd.crosstab(index=data[label1], columns=data[label2], margins=True)# Usual 2 way table b/w 2 categorical vars
  two_way['total_prop'] = two_way['normal']/two_way['normal'].loc["All","All"] #to get the total proportion of counts in each cell,
                                                                               #divide the table by the grand total
  two_way['col_prop'] = two_way['normal']/two_way['normal'].loc["All", :]
  two_way['row_prop'] = (two_way['normal'].T/two_way['normal']['All']).T
  return two_way

In [0]:
#Bivariate Cat-Cat Chi Square and Cramers'V #Used when 2 Way table > 2 X 2
def bi_cat_cat_cramerV(two_way_normal):
  observed = two_way_normal.iloc[0:-1,0:-1]
  chi_stat, p_val, df, expected = scipy.stats.chi2_contingency(observed)
  cramers_v = np.sqrt(chi_stat/(two_way_normal['All'].iloc[-1] *(min(observed.shape)-1)))
  chi2_contingency = {}
  for name in ['chi_stat','p_val','df','expected', 'cramers_v']:
    chi2_contingency[name] = eval(name)
  #chi2_contingency = dict((name,eval(name)) for name in ['chi_stat','p_val','df','expected'])
  #chi2_contingency['cramers_v'] = cramers_v
  return chi2_contingency

In [0]:
two_way_mp = bi_cat_cat_2way('Menu Header', 'Menu Item', mp)
chi2_contingency = bi_cat_cat_cramerV(two_way_mp['normal'])
print('P_Val: ',chi2_contingency['p_val'])
print('DF: ', chi2_contingency['df'])
print('Cramer\'s V: ',chi2_contingency['cramers_v'])

P_Val:  0.0
DF:  1682688
Cramer's V:  0.8965841970789699


In [0]:
#Judging 'UnKnown' in Customer ID in cbd 
cbd['Channel'].value_counts()

Zomato Online    3968
-                2713
Zomato COD        904
Swiggy Online     793
Name: Channel, dtype: int64

In [0]:
#print(cbd.loc[cbd['Channel'] == '-', 'Customer ID'].value_counts())
#When Channel = '-', people having UnKnown customer ID = 1539 ( There are 2713 total in the categ of '-' channel)
#print(cbd.loc[cbd['Customer ID'] == 'UnKnown', 'Channel'].value_counts())

cbd.loc[(cbd['Customer ID'] == 'UnKnown') & (cbd['Channel'] == '-')]['Type'].value_counts()

#list(cbd.loc[cbd['Channel'] == '-', 'Type'].index)
#list(cbd.loc[cbd['Customer ID'] == 'UnKnown', 'Type'].index)

Dine In          1361
Take Away         176
Home Delivery       2
Name: Type, dtype: int64

In [0]:
unknown_index = list(cbd.loc[cbd['Customer ID'] == 'UnKnown', 'Customer ID'].index)
print(len(cbd.loc[unknown_index]))
print(len(cbd.loc[cbd['Channel'] == '-']))
print(len(cbd.loc[(cbd['Customer ID'] == 'UnKnown') & (cbd['Channel'] == '-')]))
#The users with '-' in Channel have Customer ID as 'UnKnown'. No other channel has 'UnKnown' Customer ID.
print(cbd.loc[cbd['Customer ID'] == 'UnKnown', 'Type'].value_counts())
print(cbd.loc[cbd['Channel'] == 'Zomato Online', 'Type'].value_counts())
print(cbd.loc[cbd['Channel'] == 'Zomato COD', 'Type'].value_counts())
print(cbd.loc[cbd['Channel'] == 'Swiggy Online', 'Type'].value_counts())


1759
2713
1539
Dine In          1393
Home Delivery     184
Take Away         182
Name: Type, dtype: int64
Home Delivery    3933
Dine In            19
Take Away          16
Name: Type, dtype: int64
Home Delivery    900
Take Away          4
Name: Type, dtype: int64
Home Delivery    777
Dine In           13
Take Away          3
Name: Type, dtype: int64


In [0]:
#People having custID = Unknown and channel ! = -
cbd.loc[(cbd['Customer ID'] == 'UnKnown') & (cbd['Channel'] != '-')]['Channel'].value_counts()


Swiggy Online    191
Zomato Online     27
Zomato COD         2
Name: Channel, dtype: int64

In [0]:
#People having custID != Unknown and channel == -
cbd.loc[(cbd['Customer ID'] != 'UnKnown') & (cbd['Channel'] == '-')]['Type'].value_counts()


Take Away        749
Home Delivery    423
Dine In            2
Name: Type, dtype: int64

In [0]:
cbd.loc[(cbd['Customer ID'] == 'UnKnown') & (cbd['Channel'] == '-') & (cbd['Total Bill'] == 0)]

,Type,Date,Bill Number,Discount,Subtotal,Delivery Amt,Total Bill,Channel,Customer ID,Dish Details
486,Dine In,2018-06-08,20084093360,0.0,0.0,0,0,-,UnKnown,"Tandoori Roti (QTY-1.000),Tandoori Roti (QTY-1..."
824,Take Away,2018-06-13,20084208816,0.0,0.0,0,0,-,UnKnown,"Tandoori Chicken (QTY-1.000),Paneer Tikka (QTY..."
2213,Take Away,2018-07-05,20084679824,0.0,0.0,0,0,-,UnKnown,"Green Tea (QTY-1.000),Green Tea (QTY-1.000),Gr..."
2904,Dine In,2018-07-14,20084914016,0.0,0.0,0,0,-,UnKnown,"Chicken Garlic Tikka (QTY-1.000),water bottle ..."
4408,Dine In,2018-07-31,20085419792,0.0,0.0,0,0,-,UnKnown,"Chilly Paneer (QTY-1.000),Mushroom Masala (QTY..."
6025,Dine In,2018-08-21,20085963288,0.0,0.0,0,0,-,UnKnown,"Tandoori Roti Butter (QTY-1.000),Tandoori Roti..."


In [0]:
cbd.loc[(cbd['Customer ID'] == 'UnKnown') & (cbd['Channel'] != '-')]

,Type,Date,Bill Number,Discount,Subtotal,Delivery Amt,Total Bill,Channel,Customer ID,Dish Details
2132,Home Delivery,2018-07-04,20084652928,0.0,300.0,0,300,Zomato Online,UnKnown,Hyderabadi Mutton Dum Biryani (QTY-1.000)
3468,Home Delivery,2018-07-20,20085104584,0.0,300.0,0,300,Zomato COD,UnKnown,Hyderabadi Chicken Fry Piece Biryani (QTY-1.000)
3631,Take Away,2018-07-22,20085160016,0.0,1240.0,0,1240,Zomato Online,UnKnown,"Chilly Paneer (QTY-1.000),Tandoori Roti Butter..."
3633,Take Away,2018-07-22,20085160672,0.0,590.0,0,590,Zomato Online,UnKnown,"Tandoori Chicken (QTY-1.000),Hyderabadi Mutton..."
3634,Take Away,2018-07-22,20085161000,0.0,500.0,0,500,Zomato COD,UnKnown,"Hyderabadi Mutton Dum Biryani (QTY-1.000),Chic..."
3664,Home Delivery,2018-07-22,20085170840,0.0,350.0,0,350,Zomato Online,UnKnown,"Chicken Home Style (QTY-1.000),Kadhai Paneer (..."
3769,Take Away,2018-07-23,20085206264,0.0,280.0,0,280,Zomato Online,UnKnown,"Paneer Paratha (QTY-1.000),Paneer Paratha (QTY..."
4495,Take Away,2018-08-01,20085448984,0.0,1800.0,0,1800,Zomato Online,UnKnown,Hyderabadi Chicken Fry Piece Biryani (QTY-6.000)
4496,Take Away,2018-08-02,20085449312,0.0,895.0,0,895,Zomato Online,UnKnown,"Chicken Fried Rice (QTY-1.000),Butter nan (QTY..."
4555,Home Delivery,2018-08-02,20085469320,75.0,75.0,0,75,Zomato Online,UnKnown,Veg Biryani (QTY-1.000)


In [0]:
cbd_billno = cbd['Bill Number'].value_counts().index.tolist()
coid_billno = coid['Bill Number'].value_counts().index.tolist()

In [0]:
avoid = list(set(cbd_billno) - set(coid_billno))
avoid

[20085348288,
 20084914016,
 20085832744,
 20085419792,
 20084093360,
 20084679824,
 20084208816,
 20085963288]

In [0]:
list(set(coid_billno) - set(cbd_billno))

[]

In [0]:
cbd['Date_categ'] = cbd['Date'].apply(lambda i: i[5:7]).map({'06' : 1, '07' : 2, '08' : 3, '09' : 4})
coid['Date_categ'] = coid['Date'].apply(lambda i: i[3:5]).map({'06' : 1, '07' : 2, '08' : 3, '09' : 4})

In [0]:
index_avoid = []
for bill_no in avoid:
  index_avoid.extend(cbd.loc[cbd['Bill Number'] == bill_no, 'Bill Number'].index.tolist())

In [0]:
index_avoid

[4198, 2904, 5635, 4408, 486, 2213, 824, 6025]

In [0]:
print(cbd.loc[index_avoid, 'Channel'])
print(cbd.loc[index_avoid, 'Type'])

4198    -
2904    -
5635    -
4408    -
486     -
2213    -
824     -
6025    -
Name: Channel, dtype: object
4198    Take Away
2904      Dine In
5635    Take Away
4408      Dine In
486       Dine In
2213    Take Away
824     Take Away
6025      Dine In
Name: Type, dtype: object


In [0]:
qty_count = {}
for i in range(1, 5):
  qty_count['Dish' + str(i)] = list(coid.loc[coid['Date_categ'] == i].groupby('Dish Name').sum().index)  
  qty_count['Qty' + str(i)] = list(coid.loc[coid['Date_categ'] == i].groupby('Dish Name').sum()['Quantity'])  


In [0]:
np.argsort(qty_count['Qty1'])[-5:]

array([ 26, 145,  84, 144, 143])

In [0]:
#print(qty_count['Dish1'])
#print(qty_count['Qty1'])
#print(np.argsort(qty_count['Qty1']))
print('Min Quantity')
for i in np.argsort(qty_count['Qty1'])[0:10]:
  print(qty_count['Dish1'][i], ' ', qty_count['Qty1'][i])

print('\nMax Quantity')
for i in np.argsort(qty_count['Qty1'])[-10:]:
  print(qty_count['Dish1'][i], ' ', qty_count['Qty1'][i])



Min Quantity
Veg Roll & Biryani Combo   1
Chicken Monchow   1
Chicken Hot n Sour   1
Paneer Handi   1
Chicken Clear Soup   1
Veg Hot n Sour   1
Vegetable Cheese Half   1
Curd   1
Bread Butter Toast   1
Chicken Platter   1

Max Quantity
Hyderabadi Mutton Dum Biryani   161
Butter Chicken   170
Butter Tawa Roti   204
water bottle   207
Butter nan   228
Chicken Curry   248
Tawa Roti   264
Hyderabadi Chicken Fry Piece Biryani   383
Tandoori Roti Butter   912
Tandoori Roti   1574


In [0]:
print('Min Quantity')
for i in np.argsort(qty_count['Qty2'])[0:10]:
  print(qty_count['Dish2'][i], ' ', qty_count['Qty2'][i])

print('\nMax Quantity')
for i in np.argsort(qty_count['Qty2'])[-10:]:
  print(qty_count['Dish2'][i], ' ', qty_count['Qty2'][i])



Min Quantity
Egg Boiled Fry   1
Chicken Platter   1
Paneer Handi   1
Paneer Punjabi   1
Chicken Monchow   1
Chicken masala   1
Singapuri Noodles   1
Tomato Soup   1
Butter maggi   1
Vegetable   1

Max Quantity
Tandoori Chicken   171
Hyderabadi Mutton Dum Biryani   189
Lachchha Paratha   264
Butter Chicken   284
Butter nan   300
Chicken Curry   382
Tawa Roti   385
Hyderabadi Chicken Fry Piece Biryani   455
Tandoori Roti Butter   1021
Tandoori Roti   2293


In [0]:
print('Min Quantity')
for i in np.argsort(qty_count['Qty3'])[0:10]:
  print(qty_count['Dish3'][i], ' ', qty_count['Qty3'][i])

print('\nMax Quantity')
for i in np.argsort(qty_count['Qty3'])[-10:]:
  print(qty_count['Dish3'][i], ' ', qty_count['Qty3'][i])


Min Quantity
Tomato Soup   1
Masala Mayo Cheese Half   1
Veg Lemon Coriander   1
Mushroom Burger   1
Muttor Pulav   1
Non Veg Royal Platter   1
Masala Cheese   1
Chicken Hot n Sour   1
Curd   1
chicken masala full   1

Max Quantity
Hyderabadi Mutton Dum Biryani   161
Butter Chicken   178
Chicken Curry   220
Lachchha Paratha   228
Butter Tawa Roti   276
Butter nan   285
Hyderabadi Chicken Fry Piece Biryani   422
Tawa Roti   522
Tandoori Roti Butter   993
Tandoori Roti   1695


In [0]:
print('Min Quantity')
for i in np.argsort(qty_count['Qty4'])[0:10]:
  print(qty_count['Dish4'][i], ' ', qty_count['Qty4'][i])

print('\nMax Quantity')
for i in np.argsort(qty_count['Qty4'])[-10:]:
  print(qty_count['Dish4'][i], ' ', qty_count['Qty4'][i])


Min Quantity
Rayalaseema Kodi Kura   1
Chicken Manchurian   1
Paneer Bhurji   1
Chicken Platter   1
Chicken Rara [8 Piece]   1
Butter maggi   1
muttor paneer   1
Chicken Leg Malai 2pc   1
Veg Soup   1
Non Veg Royal Platter   1

Max Quantity
Tandoori Chicken   136
Chicken Curry   163
Lachchha Paratha   164
Butter Chicken   191
Butter nan   227
Butter Tawa Roti   244
Hyderabadi Chicken Fry Piece Biryani   291
Tawa Roti   595
Tandoori Roti Butter   702
Tandoori Roti   1441


In [0]:
print(qty_count['Dish2'])
print(qty_count['Qty2'])

['Aloo Gobhi', 'Aloo Muttor', 'Aloo Palak', 'Aloo Paratha', 'Benjo', 'Bhel', 'Bhindi masala', 'Bhuna Chicken', 'Boiled Egg', 'Boiled Fry', 'Bread Butter Toast', 'Bread Omlette', 'Bread butter cheese', 'Butter Chicken', 'Butter Khichdhi', 'Butter Milk', 'Butter Tawa Roti', 'Butter maggi', 'Butter nan', 'Cheese Chutney Half', 'Cheese Garlic Paratha', 'Cheese maggi', 'Chicken Afghani Tikka', 'Chicken Amritsari Tikka', 'Chicken Chettinad', 'Chicken Clear Soup', 'Chicken Curry', 'Chicken Fried Rice', 'Chicken Fry', 'Chicken Garlic Tikka', 'Chicken Handi', 'Chicken Home Style', 'Chicken Kolhapuri', 'Chicken Leg Afgani 2 pc', 'Chicken Leg Malai 2pc', 'Chicken Lollypop (6 pc)', 'Chicken Malai Tikka', 'Chicken Manchurian', 'Chicken Monchow', 'Chicken Nilgir', 'Chicken Pahadi Tikka', 'Chicken Palak', 'Chicken Platter', 'Chicken Pudina Tikka', 'Chicken Reshmi Kebab', 'Chicken Roast', 'Chicken Roll', 'Chicken Seekh Kebab', 'Chicken Soup', 'Chicken Tikka', 'Chicken Tikka Masala', 'Chicken masala', 

In [0]:
People fucking love Hyderabadi Chicken Fry Piece Biryani  and Chicken Curry at Golis

SyntaxError: ignored

In [0]:
coid['Dish Name'].value_counts()

Hyderabadi Chicken Fry Piece Biryani                                       1329
Tandoori Roti                                                              1262
Chicken Curry                                                               958
Butter Chicken                                                              787
Tandoori Roti Butter                                                        716
Hyderabadi Mutton Dum Biryani                                               584
Tandoori Chicken                                                            521
Egg Curry                                                                   403
Butter nan                                                                  398
Kadhai Chicken                                                              337
water bottle                                                                325
Lachchha Paratha                                                            325
Veg Biryani                             

In [0]:
mp.loc[mp['Restaurant'] == 'Goli', 'What people love here'].value_counts()

Butter Chicken, Fish Tikka, Hyderabadi Biryani, Chicken Dum Biryani, Tandoori Chicken, Chicken Tikka, Raita    181
Name: What people love here, dtype: int64

In [0]:
mp['Restaurant'].value_counts(sort = False)

CHIE    237
Kebl     87
Sint    232
Regt    201
GFCy    113
Behi     88
Noon    204
CULT    108
Foot    161
Apn)    177
Flas    317
Mr.s    624
Foos     92
Saji     15
Roti    234
Kukb    211
Last    146
Kebs    275
Galt    185
Alol    232
Line    345
12 e     76
Pini    174
Naft    104
Hotr    212
Sinb     67
Mano    168
Misf    217
Macp    164
Goli    181
Prij    314
Zaia    225
Pisa    443
Awen    118
Thei    138
Name: Restaurant, dtype: int64

In [0]:
no_love = list(mp.loc[mp['What people love here'].isnull()].index)
no_love

[308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 424,
 425,
 426,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 443,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 454,
 455,
 456,
 457,
 458,
 459,
 460,
 461,
 462,
 463,
 464,
 465,
 466,
 467,
 468,
 469,
 470,
 471,
 472,
 473,
 474

In [0]:
love_index = list(mp.loc[mp['What people love here'].notnull()].index)
love_index

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [0]:
love = list(mp['What people love here'].str.split(', ', expand=True).stack().unique())
matrix = np.zeros((len(mp), len(love)))
matrix = pd.DataFrame(matrix, columns = love)

In [0]:
for i in love_index:
  for word in mp.loc[i, 'What people love here'].split(', '):
    matrix.iloc[i, love.index(word)] = 1
  

In [0]:
matrix.head(n = 100)

,Butter Chicken,Chicken Biryani,Fish Tikka,Paneer Tikka,Dal Tadka,Gulab Jamun,Rice,Shawarma,Chicken Taka Tak,Barbeque Chicken,...,Salad,Rolls,Paneer Makhani,Malai Kofta,Lassi,Boti Chicken,Chicken Lollipop,Afghan Chicken,Chicken Malai Tikka,Mutton Rara
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
goli_dishes = dict(zip(mp.loc[mp['Restaurant'] == 'Goli', 'Menu Item'].values, mp.loc[mp['Restaurant'] == 'Goli', 'Rate'].astype(int)))
pi = {}

In [0]:
mp_temp = mp.loc[mp['Restaurant'] != 'Goli'].index.tolist()

In [0]:
(np.sum(mp.loc[mp_temp].loc[mp['Menu Item'] == 'Aloo Paratha', 'Rate'].astype(int)/goli_dishes['Aloo Paratha'])/len(mp.loc[mp_temp].loc[mp['Menu Item'] == 'Aloo Paratha', 'Rate']))*100


92.5

In [0]:
mp.loc[mp['Menu Item'] == 'Aloo Paratha', 'Rate']

1737    40
2043    40
2261    45
5313    25
6062    40
6556    35
Name: Rate, dtype: object

In [0]:
#PI value of original config is calculated above
word = 'Half bitch'
half_dishes = []
if (('half' in word.split()) | ('Half' in word.split()) | ('HALF' in word.split())):
    half_dishes.append(word)
half_dishes

['Half bitch']

In [0]:
half_dishes = []
full_dishes = []
normal_dishes = []
for dish in mp.loc[mp['Restaurant'] == 'Goli', 'Menu Item'].values:
  if (('half' in dish.split()) | ('Half' in dish.split()) | ('HALF' in dish.split())):
    half_dishes.append(dish)
  elif (('full' in dish.split()) | ('Full' in dish.split()) | ('FULL' in dish.split())):
    full_dishes.append(dish)
  else:
    normal_dishes.append(dish)

In [0]:
half_dishes_rest = []
full_dishes_rest = []
normal_dishes_rest = []
for dish in mp.loc[mp['Restaurant'] != 'Goli', 'Menu Item'].values:
  if (('half' in dish.split()) | ('Half' in dish.split()) | ('HALF' in dish.split())):
    half_dishes_rest.append(dish)
  elif (('full' in dish.split()) | ('Full' in dish.split()) | ('FULL' in dish.split())):
    full_dishes_rest.append(dish)
  else:
    normal_dishes_rest.append(dish)

In [0]:
mp.loc[mp['Restaurant'] == 'Goli', 'Menu Item'].nunique()

168

In [0]:
mp['Restaurant'].nunique()

35

In [0]:
print(len(half_dishes))
print(len(full_dishes))
print(len(normal_dishes))
print(normal_dishes)

54
54
73
['Rajma Rice Combo', 'Chole Rice Combo', 'Lemon Rice Combo [Full]', 'Veg Platter ', 'Non Veg Royal Platter', 'Chicken Platter', 'Omlatte', 'Bread Omlatte', 'Chole Bathure', 'Chole Kulche Amritsari', 'Paneer Garlic Tikka', 'Paneer Pahadi Tikka', 'Paneer Pudina Tikka', 'Chicken Roast Special', 'Chicken Leg Malai [2 Pieces]', 'Chicken Leg Afgani [2 Pieces]', 'Veg Seekh Kabab', 'Corn Cheese Kabab', 'Dal Fry', 'Dal Tadka', 'Dal Makhani', 'Sev Tamater', 'Aloo Matar', 'Jeera Aloo', 'Aloo Gobhi', 'Aloo Palak', 'Bhindi Masala', 'Mixed Veg ', 'Mushroom Masala', 'Matar Mushroom', 'Egg Curry   ', 'Kadhai Paneer ', 'Paneer Butter Masala', 'Paneer Tikka Masala', 'Palak Paneer ', 'Matar Paneer ', 'Mutton Boti', 'Tandoori Roti ', 'Butter Tandoori Roti ', 'Tandoori Paratha', 'Laccha Paratha ', 'Pudina Paratha', 'Cheese Garlic Paratha', 'Aloo Paratha', 'Gobhi Paratha', 'Paneer Paratha', 'Mixed Paratha', 'Plain Naan', 'Butter Naan ', 'Garlic Naan', 'Veg Pulav', 'Matar Pulav', 'Tadka Khichdi', 'V

In [0]:
half_restaur = {}
full_restaur = {}
for half in half_dishes:
  half_restaur[half] = mp.loc[mp_temp].loc[mp['Menu Item'] == half, 'Restaurant']
for full in full_dishes:
  full_restaur[full] = mp.loc[mp_temp].loc[mp['Menu Item'] == full, 'Restaurant']

In [0]:
half_restaur['Tandoori Chicken Half']

14      12 e
339     Apn)
2876    Last
2909    Last
5832    Regt
Name: Restaurant, dtype: object

In [0]:
half_dishes_rest

['Tandoori Chicken Half',
 'Tandoori Chicken Half',
 'Garlic Tandoori Chicken Half',
 'Chicken Kali Mirch Half',
 'Mutton Rogan Josh Half',
 'Mutton Masala Half',
 'Bhuna Mutton Half',
 'Rara Mutton Half',
 'Mutton Curry Wala Half',
 'Fish Curry Half',
 'Egg Biryani Half',
 'Chicken Biryani Half',
 'Hyderabadi Murg Biryani Half',
 'Hyderabadi Murg Biryani Half',
 'Mutton Biryani Half',
 'HALF FRY',
 'BUTTER CHICKEN HALF',
 'Tandoori Chicken Full / Half',
 'Butter Chicken Full / Half',
 'Tandoori Murgh Half',
 'Tandoori Murgh Half',
 'Chicken Lollypop Half',
 "Hyderabadi Biryani [Chef's Special] Served with mirch ka salan. half",
 'Butter Chicken half',
 "Tawa Chicken [Chef's Special] half",
 'Kadhai Chicken half',
 'Chicken Korma half',
 'Chicken Do Pyaza half',
 'Green Chicken half',
 'Chicken Rara half',
 'Chicken Guldasta half',
 'Egg Half Fry [2 Egg]',
 'Tandoori Chicken Half',
 'Chicken Curry Half [2 Pieces]',
 'Tandoori Chicken Half',
 'Chicken Curry Half [2 Pieces]',
 'Mutton Cu

In [0]:
mp.loc[mp['Restaurant'] == 'Goli'].index 

Int64Index([1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913,
            ...
            2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084],
           dtype='int64', length=181)

In [0]:
temp = []
for restaur in mp['Restaurant'].value_counts().keys().tolist():
  temp.append(list(mp.loc[mp['Restaurant'] == restaur].index)[0])

In [0]:
temp = np.sort(temp)
temp

array([   0,   76,  308,  485,  603,  691,  928, 1036, 1353, 1445, 1606,
       1791, 1904, 2085, 2297, 2572, 2659, 2870, 3016, 3361, 3525, 3693,
       3910, 4534, 4638, 4842, 5016, 5459, 5773, 5974, 6208, 6223, 6455,
       6522, 6660])

In [0]:
matrix = matrix.loc[temp]

In [0]:
matrix.sum()

Butter Chicken           11.0
Chicken Biryani           6.0
Fish Tikka                2.0
Paneer Tikka              1.0
Dal Tadka                 1.0
Gulab Jamun               2.0
Rice                      5.0
Shawarma                  2.0
Chicken Taka Tak          1.0
Barbeque Chicken          1.0
Chicken Grill             1.0
Chicken Tenders           1.0
Chicken Shawarama         1.0
Hyderabadi Chicken        1.0
Dal Makhani               4.0
Nalli Nihari              1.0
Tandoori Chicken          9.0
Dum Biryani               2.0
Biryani                   2.0
Panneer Butter Masala     2.0
Naan                      4.0
Hyderabadi Biryani        1.0
Chicken Dum Biryani       1.0
Chicken Tikka             5.0
Raita                     1.0
Rumali Roti               3.0
Chicken Tikka Masala      1.0
Jumbo Prawns              1.0
Paan Ice Cream            1.0
Salads                    1.0
                         ... 
Keema Kaleji              1.0
Mutton Bhuna              1.0
Mutton Cur

In [0]:
matrix.sum()
preference = dict(zip(list(matrix.sum().keys()), list(matrix.sum().values)))

In [0]:
preference

{'Afghan Chicken': 1.0,
 'Barbeque Chicken': 1.0,
 'Biryani': 2.0,
 'Boti Chicken': 1.0,
 'Butter Chicken': 11.0,
 'Butter Naan': 1.0,
 'Chicken Biryani': 6.0,
 'Chicken Dum Biryani': 1.0,
 'Chicken Grill': 1.0,
 'Chicken Lollipop': 1.0,
 'Chicken Malai Tikka': 1.0,
 'Chicken Roll': 1.0,
 'Chicken Shawarama': 1.0,
 'Chicken Sukha': 1.0,
 'Chicken Taka Tak': 1.0,
 'Chicken Tariwala': 1.0,
 'Chicken Tenders': 1.0,
 'Chicken Tikka': 5.0,
 'Chicken Tikka Biryani': 1.0,
 'Chicken Tikka Masala': 1.0,
 'Crispy Corn': 1.0,
 'Dahi Kebab': 1.0,
 'Dal Makhani': 4.0,
 'Dal Tadka': 1.0,
 'Dum Biryani': 2.0,
 'Egg Curry': 1.0,
 'Fish': 2.0,
 'Fish Tikka': 2.0,
 'Gulab Jamun': 2.0,
 'Haleem': 1.0,
 'Hyderabadi Biryani': 1.0,
 'Hyderabadi Chicken': 1.0,
 'Jumbo Prawns': 1.0,
 'Keema Kaleji': 1.0,
 'Kiwi Mojito': 1.0,
 'Lassi': 1.0,
 'Malai Kofta': 1.0,
 'Manchow Soup': 1.0,
 'Mint Mojito': 1.0,
 'Mutton Bhuna': 1.0,
 'Mutton Biryani': 2.0,
 'Mutton Curry': 1.0,
 'Mutton Rara': 1.0,
 'Naan': 4.0,
 'Nal

In [0]:
list(matrix.sum().values)
matrix.sum()['Tandoori Chicken']

9.0

In [0]:
[x for _,x in sorted(zip(list(matrix.sum().values),list(matrix.sum().keys())))]

['Afghan Chicken',
 'Barbeque Chicken',
 'Boti Chicken',
 'Butter Naan',
 'Chicken Dum Biryani',
 'Chicken Grill',
 'Chicken Lollipop',
 'Chicken Malai Tikka',
 'Chicken Roll',
 'Chicken Shawarama',
 'Chicken Sukha',
 'Chicken Taka Tak',
 'Chicken Tariwala',
 'Chicken Tenders',
 'Chicken Tikka Biryani',
 'Chicken Tikka Masala',
 'Crispy Corn',
 'Dahi Kebab',
 'Dal Tadka',
 'Egg Curry',
 'Haleem',
 'Hyderabadi Biryani',
 'Hyderabadi Chicken',
 'Jumbo Prawns',
 'Keema Kaleji',
 'Kiwi Mojito',
 'Lassi',
 'Malai Kofta',
 'Manchow Soup',
 'Mint Mojito',
 'Mutton Bhuna',
 'Mutton Curry',
 'Mutton Rara',
 'Nalli Nihari',
 'Paan Ice Cream',
 'Paneer Makhani',
 'Paneer Tikka',
 'Raita',
 'Rolls',
 'Roti',
 'Salad',
 'Salads',
 'Seekh Kebab',
 'Vegetable Biryani',
 'Biryani',
 'Dum Biryani',
 'Fish',
 'Fish Tikka',
 'Gulab Jamun',
 'Mutton Biryani',
 'Paneer Tikka Masala',
 'Panneer Butter Masala',
 'Shawarma',
 'Rumali Roti',
 'Dal Makhani',
 'Naan',
 'Chicken Tikka',
 'Rice',
 'Chicken Biryani

In [0]:
preference = dict(zip([x for _,x in sorted(zip(list(matrix.sum().values),list(matrix.sum().keys())))], np.sort(list(matrix.sum().values))))

In [0]:
preference

{'Afghan Chicken': 1.0,
 'Barbeque Chicken': 1.0,
 'Biryani': 2.0,
 'Boti Chicken': 1.0,
 'Butter Chicken': 11.0,
 'Butter Naan': 1.0,
 'Chicken Biryani': 6.0,
 'Chicken Dum Biryani': 1.0,
 'Chicken Grill': 1.0,
 'Chicken Lollipop': 1.0,
 'Chicken Malai Tikka': 1.0,
 'Chicken Roll': 1.0,
 'Chicken Shawarama': 1.0,
 'Chicken Sukha': 1.0,
 'Chicken Taka Tak': 1.0,
 'Chicken Tariwala': 1.0,
 'Chicken Tenders': 1.0,
 'Chicken Tikka': 5.0,
 'Chicken Tikka Biryani': 1.0,
 'Chicken Tikka Masala': 1.0,
 'Crispy Corn': 1.0,
 'Dahi Kebab': 1.0,
 'Dal Makhani': 4.0,
 'Dal Tadka': 1.0,
 'Dum Biryani': 2.0,
 'Egg Curry': 1.0,
 'Fish': 2.0,
 'Fish Tikka': 2.0,
 'Gulab Jamun': 2.0,
 'Haleem': 1.0,
 'Hyderabadi Biryani': 1.0,
 'Hyderabadi Chicken': 1.0,
 'Jumbo Prawns': 1.0,
 'Keema Kaleji': 1.0,
 'Kiwi Mojito': 1.0,
 'Lassi': 1.0,
 'Malai Kofta': 1.0,
 'Manchow Soup': 1.0,
 'Mint Mojito': 1.0,
 'Mutton Bhuna': 1.0,
 'Mutton Biryani': 2.0,
 'Mutton Curry': 1.0,
 'Mutton Rara': 1.0,
 'Naan': 4.0,
 'Nal

In [0]:
coid.groupby(by = 'Dish Name').sum().sort_values(axis = 0, by = 'Quantity', ascending = False)

,Bill Number,Quantity,Date_categ
Dish Name,,,
Tandoori Roti,27697407089888,7003,3022
Tandoori Roti Butter,15465557998576,3628,1702
Tawa Roti,6065705414776,1766,660
Hyderabadi Chicken Fry Piece Biryani,29585397187816,1551,3198
Butter nan,8716971576960,1040,977
Chicken Curry,21229904545496,1013,2196
Butter Tawa Roti,3173439439720,825,327
Butter Chicken,17795424687552,823,1946
Lachchha Paratha,7069979479128,816,785
